# Setup

## Requirements

In [ ]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0


## API Setup

In [1]:
import os
import json

with open('config.json', 'r') as file:
    config = json.load(file)
    togetherAI_api_key = config["togetherAI_api_key"]
    OPEN_AI_API_KEY = config["OpenAI_api_key"]
    weather_api_key = config["weather"]
    XAI_API_KEY = config["grok"]

os.environ['TOGETHER_API_KEY'] = togetherAI_api_key

In [5]:
from openai import OpenAI


# together.ai API Using OpenAI client
client_together = OpenAI(
  api_key=os.environ["TOGETHER_API_KEY"],
  base_url="https://api.together.xyz/v1"
)

# together.ai API Using together.ai client
# from together import Together
# client_together = Together()

# OpenAI API Using OpenAI client
from openai import OpenAI
client_openai = OpenAI(
  api_key=OPEN_AI_API_KEY
)

# grok
# client_grok = OpenAI(
#   api_key=XAI_API_KEY,
#   base_url="https://api.x.ai/v1",
# )


## SQL Database

In [2]:
sql_schema = """CREATE TABLE WeatherData (
    Region VARCHAR(255),
    Country VARCHAR(255),
    State VARCHAR(255),
    City VARCHAR(255),
    Month INT,
    Day INT,
    Year INT,
    Temperature FLOAT
);
"""

# Approaches

## Before Structured Output

In [6]:
question = "give me country names list with average temperature less than 30"

extract = client_together.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"The following is a question about this SQL database: {sql_schema}. generate a sql query to answer users question.",
        },
        {
            "role": "user",
            "content": question,
        },
    ],
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0.7,
)

result = extract
print(result.choices[0].message.content)

To get the list of country names with an average temperature less than 30, you can use the following SQL query:

```sql
SELECT Country
FROM WeatherData
GROUP BY Country
HAVING AVG(Temperature) < 30;
```

This query works as follows:

- `GROUP BY Country` groups the rows by country, allowing us to calculate aggregate values for each country.
- `HAVING AVG(Temperature) < 30` filters these groups to only include countries where the average temperature is less than 30.

Note that this query assumes that there are temperature readings for each country. If a country has no temperature readings, it will not be included in the results.


In [7]:
print(result.usage)

CompletionUsage(completion_tokens=139, prompt_tokens=113, total_tokens=252, completion_tokens_details=None, prompt_tokens_details=None)


## Simple Prompt

In [14]:
from pydantic import BaseModel, Field

class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")




def simple_prompt(question):
    extract = client_together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. generate a sql query to answer users question.",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": TextToSQLResult.model_json_schema(),
        },
        temperature=0,

    )
    # print(extract.choices[0].message.model_dump())
    output = json.loads(extract.choices[0].message.content)
    return output


question = "give me country names list with at least one city with temperature less than 30"
question = "give me country names list with average temperature less than 30"
question = "give me country names list with temperature less than 30"


# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = simple_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

{
  "generated_query": "SELECT Country FROM WeatherData WHERE Temperature < 30"
}


## Few Shot

In [12]:
from pydantic import BaseModel, Field

class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")


few_shot_system_prompt = """
The following is a question about this SQL database: {}.
generate a sql query to answer users question.

When answering user question follow these examples:
Question: give me country names list with temperature less than 30?
Answer: SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 30.
Question: give me country names list with average temperature less than 30?
Answer: SELECT Country FROM WeatherData GROUP BY Country HAVING AVG(Temperature) < 30.
"""

def few_shot_prompt(question):
    extract = client_together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": few_shot_system_prompt.format(sql_schema),
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": TextToSQLResult.model_json_schema(),
        },
        temperature=0,

    )

    output = json.loads(extract.choices[0].message.content)
    return output


question = "give me country names list with at least one city with temperature less than 30"
question = "give me country names list with temperature less than 20"

# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = few_shot_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

{
  "generated_query": "SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 20"
}


## CoT

### Cot create

In [13]:
from pydantic import BaseModel, Field
from typing import List

class Step(BaseModel):
    explanation: str
    current_question: str = Field("rewrote question")

class TextToSQLReasoning(BaseModel):
    steps: List[Step] = Field(description="Reasoning Steps to get to final SQL Query")
    generated_query: str = Field(description="Generated SQL Query for the question.")




def CoT_prompt(question):
    extract = client_together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. generate a sql query to answer users question. make your output step by step",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": TextToSQLReasoning.model_json_schema(),
        },
        temperature=0,

    )

    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
question = "give me country names list with temperature less than 30"

# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = CoT_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

{
  "steps": [
    {
      "explanation": "To get the list of country names with a temperature less than 30, we need to select the `Country` column from the `WeatherData` table.",
      "current_question": "SELECT Country FROM WeatherData"
    },
    {
      "explanation": "Next, we need to add a condition to filter the results to only include rows where the temperature is less than 30. We can do this using the `WHERE` clause.",
      "current_question": "SELECT Country FROM WeatherData WHERE Temperature < 30"
    },
    {
      "explanation": "Finally, to get a list of unique country names, we can use the `DISTINCT` keyword.",
      "current_question": "SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 30"
    }
  ],
  "generated_query": "SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 30"
}


### CoT parse

In [ ]:
# client_together.beta.chat.completions.parse??

In [15]:
from pydantic import BaseModel, Field
from typing import List

class Step(BaseModel):
    explanation: str
    current_question: str = Field("rewrote question")

class TextToSQLReasoning(BaseModel):
    steps: List[Step] = Field(description="Reasoning Steps to get to final SQL Query")
    generated_query: str = Field(description="Generated SQL Query for the question.")




def CoT_prompt(question):
    extract = client_together.beta.chat.completions.parse(
        messages=[
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. generate a sql query to answer users question. make your output step by step",
            },
            {
                "role": "user",
                "content": question,
            },
        ],
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": TextToSQLReasoning.model_json_schema(),
        },
        temperature=0,

    )

    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
question = "give me country names list with temperature less than 30"

# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = CoT_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

{
  "steps": [
    {
      "explanation": "To get the list of country names with a temperature less than 30, we need to select the `Country` column from the `WeatherData` table.",
      "current_question": "SELECT Country FROM WeatherData"
    },
    {
      "explanation": "Next, we need to add a condition to filter the results to only include rows where the temperature is less than 30. We can do this using the `WHERE` clause.",
      "current_question": "SELECT Country FROM WeatherData WHERE Temperature < 30"
    },
    {
      "explanation": "Finally, to get a list of unique country names, we can use the `DISTINCT` keyword.",
      "current_question": "SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 30"
    }
  ],
  "generated_query": "SELECT DISTINCT Country FROM WeatherData WHERE Temperature < 30"
}


## Critique Agent

In [16]:
from pydantic import BaseModel, Field
from typing import List, Literal
import time

# class Step(BaseModel):
#     explanation: str
#     current_question: str = Field("rewrote question")

# class TextToSQLReasoning(BaseModel):
#     steps: List[Step] = Field(description="Reasoning Steps to get to final SQL Query")
#     generated_query: str = Field(description="Generated SQL Query for the question.")


class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")


class Critique(BaseModel):
    feedback: str = Field(description="Feedback about the generated query.")
    is_ok: Literal[True, False] = Field(description="Does the query exactly generate the desired output and has nothing to fix?")

def inference(messages, schema_class):
    extract = client_together.chat.completions.create(
        messages=messages,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": schema_class.model_json_schema(),
        },
        temperature=0,
    )
    return extract

def critique_prompt(question):
    messages = [
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. generate a sql query to answer users question.",
            },
            {
                "role": "user",
                "content": question,
            },
        ]
    extract = inference(messages, TextToSQLResult)

    initial_query_message = extract.choices[0].message

    critique_system_prompt = {
                "role": "system",
                "content": "You are a critical thinker and Professional SQL Expert. Give feedback about the generated sql query for the user question. if you find something that should be changed also set is_ok to false",
            }

    messages.append({"role": "assistant", "content": initial_query_message.content})
    messages.append(critique_system_prompt)

    extract = inference(messages, Critique)
    feedback_query_message = extract.choices[0].message

    query_rewrite_system_prompt = {
                "role": "user",
                "content": "You are a humble and professional SQL query expert. according to the question and last generated query and the professional feedback rewrite the query",
            }

    messages.append({"role": "assistant", "content": feedback_query_message.content})
    messages.append(query_rewrite_system_prompt)


    print("\n".join([str(i) for i in messages]))

    extract = inference(messages, TextToSQLResult)

    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
question = "give me country names list with temperature less than 30"

# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = critique_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

{'role': 'system', 'content': 'The following is a question about this SQL database: CREATE TABLE WeatherData (\n    Region VARCHAR(255),\n    Country VARCHAR(255),\n    State VARCHAR(255),\n    City VARCHAR(255),\n    Month INT,\n    Day INT,\n    Year INT,\n    Temperature FLOAT\n);\n. generate a sql query to answer users question.'}
{'role': 'user', 'content': 'give me country names list with temperature less than 30'}
{'role': 'assistant', 'content': '{ "generated_query": "SELECT Country FROM WeatherData WHERE Temperature < 30" }'}
{'role': 'system', 'content': 'You are a critical thinker and Professional SQL Expert. Give feedback about the generated sql query for the user question. if you find something that should be changed also set is_ok to false'}
{'role': 'assistant', 'content': '{ "feedback": "The generated query is mostly correct, but it may return duplicate country names if there are multiple records for the same country with a temperature less than 30. To get a list of uni

## Weather API Tool Use

In [17]:
import requests

def get_weather(city_name):
    api_key = weather_api_key  # Replace with your OpenWeatherMap API key
    api_url = f'http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric'

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        weather_data = response.json()
        # weather = {
        #     'temperature': weather_data['main']['temp'],
        #     'description': weather_data['weather'][0]['description'],
        #     'humidity': weather_data['main']['humidity'],
        #     'wind_speed': weather_data['wind']['speed']
        # }
        weather = {
            'city_name': city_name,
            'temperature': weather_data['main']['temp']
        }
        return weather
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    return None

# Example usage:
weather = get_weather('tehran')
if weather:
    print(weather)


{'city_name': 'tehran', 'temperature': 7.62}


In [18]:
class get_weather_class(BaseModel):
    city_name: str

### TogetherAI Llama Response Format + Tool Use At the same time ❌

In [ ]:
get_weather_tool_json

{'type': 'function',
 'function': {'name': 'get_weather_class',
  'strict': True,
  'parameters': {'properties': {'city_name': {'title': 'City Name',
     'type': 'string'}},
   'required': ['city_name'],
   'title': 'get_weather_class',
   'type': 'object',
   'additionalProperties': False}}}

In [ ]:
get_weather_tool_json = pydantic_function_tool(get_weather_class)


tools = [
  {'type': 'function',
 'function': {'name': 'get_weather_class',
  'strict': True,
  'parameters': {'properties': {'city_name': {'title': 'City Name',
     'type': 'string'}},
   'required': ['city_name'],
   'title': 'get_weather_class',
   'type': 'object',
   'additionalProperties': False}}}
]

from pydantic import BaseModel, Field
from typing import List, Literal
import time


class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")


def tool_use_prompt(question):
    messages = [
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. if there is no need to live weather data generate a sql query to answer users question. otherwise suggest using live weather api.",
            },
            {
                "role": "user",
                "content": question,
            },
        ]

    extract = client_together.beta.chat.completions.parse(
        messages=messages,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format=TextToSQLResult,
        tools=tools
    )

    print(extract.choices[0].message)

    if extract.choices[0].message.tool_calls:
        messages.append(extract.choices[0].message)
        tool_call = extract.choices[0].message.tool_calls[0]
        # print(tool_call)
        # print(tool_call.function)
        # print(tool_call.function.name)
        # print(tool_call.function.arguments)
        functions = {"get_weather": get_weather}
        right_function = functions[tool_call.function.name]
        tool_result = right_function(**json.loads(tool_call.function.arguments))
        print("Tool Use")
        print(tool_result)
        messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": f"{tool_result['city_name']}'s Temperature is {tool_result['temperature']}"})
        print("\n".join([str(message) for message in messages]))

        extract = client_together.beta.chat.completions.parse(
            messages=messages,
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
            response_format=TextToSQLResult,
            temperature=0
        )
    else:
        print("No Tool Use")



    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
# question = "how is the weather in tehran right now?"
# question = "give me country names list with temperature less than 30"
question = "give me country names list with temperature less than tehran's current temperature"


# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = tool_use_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))
print("-" * 50)
print(output["generated_query"])

InternalServerError: Error code: 500 - {'error': {'message': 'Internal server error', 'type': 'server_error', 'param': None, 'code': None}}

{'properties': {'generated_query': {'description': 'Generated SQL Query for the question.',
   'title': 'Generated Query',
   'type': 'string'}},
 'required': ['generated_query'],
 'title': 'TextToSQLResult',
 'type': 'object'}

### OpenAI ✅

In [21]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "strict": True,
          "parameters": {
              "type": "object",
              "properties": {
                  "city_name": {"type": "string"}
              },
            "required": ["city_name"],
            "additionalProperties": False
          }
      }
  }
]

from pydantic import BaseModel, Field
from typing import List, Literal
import time


class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")


def tool_use_prompt(question):
    messages = [
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. if there is no need to live weather data generate a sql query to answer users question. otherwise suggest using live weather api.",
            },
            {
                "role": "user",
                "content": question,
            },
        ]

    extract = client_openai.beta.chat.completions.parse(
        messages=messages,
        model="gpt-4o-mini",
        response_format=TextToSQLResult,
        temperature=0,
        tools=tools
    )

    print(extract.choices[0].message)

    if extract.choices[0].message.tool_calls:
        messages.append(extract.choices[0].message)
        tool_call = extract.choices[0].message.tool_calls[0]
        # print(tool_call)
        # print(tool_call.function)
        # print(tool_call.function.name)
        # print(tool_call.function.arguments)
        functions = {"get_weather": get_weather}
        right_function = functions[tool_call.function.name]
        tool_result = right_function(**json.loads(tool_call.function.arguments))
        print("Tool Use")
        print(tool_result)
        messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": f"{tool_result['city_name']}'s Temperature is {tool_result['temperature']}"})
        print("\n".join([str(message) for message in messages]))
        extract = client_openai.beta.chat.completions.parse(
            messages=messages,
            model="gpt-4o-mini",
            response_format=TextToSQLResult,
            temperature=0
        )
    else:
        print("No Tool Use")



    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
# question = "how is the weather in tehran right now?"
# question = "give me country names list with temperature less than 30"
question = "give me country names list with temperature less than tehran's current temperature"


# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = tool_use_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))
print("-" * 50)
print(output["generated_query"])

ParsedChatCompletionMessage[TextToSQLResult](content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ParsedFunctionToolCall(id='call_TNEhZlgkCrU1ncwDuk2zMvgd', function=ParsedFunction(arguments='{"city_name":"Tehran"}', name='get_weather', parsed_arguments={'city_name': 'Tehran'}), type='function')], parsed=None)
Tool Use
{'city_name': 'Tehran', 'temperature': 7.62}
{'role': 'system', 'content': 'The following is a question about this SQL database: CREATE TABLE WeatherData (\n    Region VARCHAR(255),\n    Country VARCHAR(255),\n    State VARCHAR(255),\n    City VARCHAR(255),\n    Month INT,\n    Day INT,\n    Year INT,\n    Temperature FLOAT\n);\n. if there is no need to live weather data generate a sql query to answer users question. otherwise suggest using live weather api.'}
{'role': 'user', 'content': "give me country names list with temperature less than tehran's current temperature"}
ParsedChatCompletionMessage[TextToSQLResult](content=None, refu

### TogetherAI Tool Use As Response Format ✅

In [22]:
from openai import pydantic_function_tool

In [23]:
TextToSQLResult.model_json_schema()

{'properties': {'generated_query': {'description': 'Generated SQL Query for the question.',
   'title': 'Generated Query',
   'type': 'string'}},
 'required': ['generated_query'],
 'title': 'TextToSQLResult',
 'type': 'object'}

In [24]:
get_weather_tool_json = pydantic_function_tool(get_weather_class)
get_weather_tool_json

{'type': 'function',
 'function': {'name': 'get_weather_class',
  'strict': True,
  'parameters': {'properties': {'city_name': {'title': 'City Name',
     'type': 'string'}},
   'required': ['city_name'],
   'title': 'get_weather_class',
   'type': 'object',
   'additionalProperties': False}}}

In [ ]:
# {
#       "type": "function",
#       "function": {
#           "name": "get_weather",
#           "strict": True,
#           "parameters": {
#               "type": "object",
#               "properties": {
#                   "city_name": {"type": "string"}
#               },
#             "required": ["city_name"],
#             "additionalProperties": False
#           },
#       },
#   },

In [25]:
text_to_sql_result_json = pydantic_function_tool(TextToSQLResult)
text_to_sql_result_json

{'type': 'function',
 'function': {'name': 'TextToSQLResult',
  'strict': True,
  'parameters': {'properties': {'generated_query': {'description': 'Generated SQL Query for the question.',
     'title': 'Generated Query',
     'type': 'string'}},
   'required': ['generated_query'],
   'title': 'TextToSQLResult',
   'type': 'object',
   'additionalProperties': False}}}

In [33]:
from pydantic import BaseModel, Field
from typing import List, Literal
import time

class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")

tools = [
    get_weather_tool_json,
    text_to_sql_result_json
]



def tool_use_prompt(question):
    messages = [
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. if we need live weather data for generating the query suggest using weather api.",
            },
            {
                "role": "user",
                "content": question,
            },
        ]

    extract = client_together.chat.completions.create(
        messages=messages,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        temperature=0,
        tools=tools
    )
    print(extract.choices[0].message)

    if extract.choices[0].message.tool_calls:
        tool_call = extract.choices[0].message.tool_calls[0]
        if tool_call.function.name == "TextToSQLResult":
            print("Direct Output")
            print(tool_call)
            output = json.loads(tool_call.function.arguments)
        elif tool_call.function.name == "get_weather_class":
            messages.append(extract.choices[0].message)

            functions = {"get_weather_class": get_weather}
            right_function = functions[tool_call.function.name]
            tool_result = right_function(**json.loads(tool_call.function.arguments))
            print("Tool Use")
            # print(tool_result)
            messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": f"{tool_result['city_name']}'s Temperature is {tool_result['temperature']}"})
            print("\n".join([str(message) for message in messages]))
            extract = client_together.chat.completions.create(
                messages=messages,
                model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                response_format={"type":"json_object" , "schema": TextToSQLResult.model_json_schema()},
                temperature=0
            )
            output = extract.choices[0].message.content
    else:
        print("Something Went Wrong")


    return output


# question = "give me country names list with at least one city with temperature less than 30"
# question = "how is the weather in tehran right now?"
# question = "give me country names list with temperature less than 30"
question = "give me country names list with temperature less than tehran's current temperature"


# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = tool_use_prompt(question)
print("-" * 25)
print(json.dumps(output, indent=2, ensure_ascii=False))
# print(output)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hojdw0xh0rmgcsp9pcedws8b', function=Function(arguments='{"city_name":"Tehran"}', name='get_weather_class'), type='function', index=0)])
Tool Use
{'role': 'system', 'content': 'The following is a question about this SQL database: CREATE TABLE WeatherData (\n    Region VARCHAR(255),\n    Country VARCHAR(255),\n    State VARCHAR(255),\n    City VARCHAR(255),\n    Month INT,\n    Day INT,\n    Year INT,\n    Temperature FLOAT\n);\n. if we need live weather data for generating the query suggest using weather api.'}
{'role': 'user', 'content': "give me country names list with temperature less than tehran's current temperature"}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hojdw0xh0rmgcsp9pcedws8b', function=Function(arguments='{"city_name"

### TogetherAI Two LLM Calls ✅

In [35]:
from pydantic import BaseModel, Field
from typing import List, Literal
import time

tools = [
  get_weather_tool_json
]


class TextToSQLResult(BaseModel):
    generated_query: str = Field(description="Generated SQL Query for the question.")


def tool_use_prompt(question):
    messages = [
            {
                "role": "system",
                "content": f"The following is a question about this SQL database: {sql_schema}. if there is no need to live weather data generate a sql query to answer users question. otherwise suggest using live weather api.",
            },
            {
                "role": "user",
                "content": question,
            },
        ]

    extract = client_together.chat.completions.create(
        messages=messages,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        # response_format={
        #     "type": "json_object",
        #     "schema": TextToSQLResult.model_json_schema(),
        # },
        temperature=0,
        tools=tools
    )
    print(extract.choices[0].message)

    if extract.choices[0].message.tool_calls:
        messages.append(extract.choices[0].message)
        tool_call = extract.choices[0].message.tool_calls[0]
        # print(tool_call)
        # print(tool_call.function)
        # print(tool_call.function.name)
        # print(tool_call.function.arguments)
        functions = {"get_weather_class": get_weather}
        right_function = functions[tool_call.function.name]
        tool_result = right_function(**json.loads(tool_call.function.arguments))
        print("Tool Use")
        print(tool_result)
        messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": f"Note: {tool_result['city_name']}'s Temperature is {tool_result['temperature']}"})
    else:
        print("No Tool Use")
    print("\n".join([str(message) for message in messages]))
    extract = client_together.chat.completions.create(
        messages=messages,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": TextToSQLResult.model_json_schema(),
        },
        temperature=0
    )

    output = json.loads(extract.choices[0].message.content)
    return output


# question = "give me country names list with at least one city with temperature less than 30"
# question = "how is the weather in tehran right now?"
# question = "give me country names list with temperature less than 30"
question = "give me country names list with temperature less than tehran's current temperature"


# question = "give me country names list with average temperature less than 30"
# question = "give me country names list with temperature less than 30 in same year as 2000-08-28"

# question = "give me country names with average temperature less than 30 and number of cities more than 5"
# question = "give me city names with temperature less than 30 in Australia or US"

output = tool_use_prompt(question)
print(json.dumps(output, indent=2, ensure_ascii=False))

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5k3qzx1zfv1se8s00cuwt72c', function=Function(arguments='{"city_name":"Tehran"}', name='get_weather_class'), type='function', index=0)])
Tool Use
{'city_name': 'Tehran', 'temperature': 8.73}
{'role': 'system', 'content': 'The following is a question about this SQL database: CREATE TABLE WeatherData (\n    Region VARCHAR(255),\n    Country VARCHAR(255),\n    State VARCHAR(255),\n    City VARCHAR(255),\n    Month INT,\n    Day INT,\n    Year INT,\n    Temperature FLOAT\n);\n. if there is no need to live weather data generate a sql query to answer users question. otherwise suggest using live weather api.'}
{'role': 'user', 'content': "give me country names list with temperature less than tehran's current temperature"}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletio